### Simple notebook that calculates the model performance metrics
The three metrcis calculated are:
* Accuracy,
* Jensen-Shannon divergence, and
* Kullback–Leibler divergence

In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import jensenshannon
from scipy.stats import entropy

### Helper functions

In [2]:
# Note: This comes from the ChaosNLI github repository:
# https://github.com/easonnie/ChaosNLI/blob/master/distnli/src/evaluation/tools.py#L24
def model_label_dist(logits_list):

    logits = np.asarray(logits_list)
    prob = np.exp(logits_list) / np.sum(np.exp(logits_list))

    # numerical stability for KL
    for i, value in enumerate(prob):
        if np.abs(value) < 1e-15:
            prob[i] = 1e-15
    
    # normalize
    prob = prob / np.sum(prob)
    assert np.isclose(np.sum(prob), 1)
    
    return prob


def generate_metrics_dict(df):
    
    return {'acc': len(df.query('label_g == predicted_label'))/len(df),
            'jsd': df.jsd.mean(),
            'kld': df.kld.mean()}


def generate_all_metrics_dict(df):

    return {'all': generate_metrics_dict(eval_df),
            'no-ambiguity': generate_metrics_dict(eval_df.query('amb_level == 0')),
            'medium-ambiguity': generate_metrics_dict(eval_df.query('amb_level == 1')),
            'high-ambiguity': generate_metrics_dict(eval_df.query('amb_level == 2'))}


def calculate_example_metrics(df):

    df['predicted_probs'] = df.apply(lambda x: model_label_dist(x.predicted_scores), axis=1)
    df['jsd'] = df.apply(lambda x: jensenshannon(x.label, x.predicted_probs), axis=1)
    df['kld'] = df.apply(lambda x: entropy(x.label, x.predicted_probs), axis=1)
    df['amb_level'] = df.apply(lambda x: get_ambiguity_level(x), axis=1)

    return df


def get_ambiguity_level(x):

    # this whole thing is a bit klugy, but <shrug> it works since there might be some fp imprecision.
    label_count = max(x.label)*5

    if label_count > 4.5:
        return 0
    elif label_count > 3.5:
        return 1
    else:
        return 2

In [8]:
BASE_OUTPUT_PATH = '/Users/richardross/workspace/msds/dsc-395t-nlp-final-project/fp-dataset-artifacts/output_final'

### Evaluate the baseline performance
This model has been trained on the training dataset using gold labels, fine tuned on the dev dataset using gold labels, and evaluated on the test dataset using the probability distributions.

In [13]:
eval_df = pd.read_json(f'{BASE_OUTPUT_PATH}/trained_model_snli_baseline/trained_train_gold_dev_gold_evaled_test_probs/eval_predictions.jsonl', lines=True)
eval_df = calculate_example_metrics(eval_df)
generate_all_metrics_dict(eval_df)

{'all': {'acc': 0.8890472312703583,
  'jsd': 0.19018686160110934,
  'kld': 0.42401248721189044},
 'no-ambiguity': {'acc': 0.9544101228135468,
  'jsd': 0.10453239072285646,
  'kld': 0.135539159308617},
 'medium-ambiguity': {'acc': 0.8847091605712295,
  'jsd': 0.2620504238347566,
  'kld': 0.6118892730848189},
 'high-ambiguity': {'acc': 0.6744775174160861,
  'jsd': 0.35103983140917144,
  'kld': 1.0642033750592708}}

### Evaluate the baseline performance
This model has been trained on the training dataset using gold labels and evaluated on the test dataset using the probability distributions (no fine-tuning).

In [14]:
eval_df = pd.read_json(f'{BASE_OUTPUT_PATH}/trained_model_snli_baseline/trained_train_gold_eval_test_probs/eval_predictions.jsonl', lines=True)
eval_df = calculate_example_metrics(eval_df)
generate_all_metrics_dict(eval_df)

{'all': {'acc': 0.8897597719869706,
  'jsd': 0.1961546240063928,
  'kld': 0.3613578620539617},
 'no-ambiguity': {'acc': 0.9531075548939337,
  'jsd': 0.12868704955131746,
  'kld': 0.1388770659038094},
 'medium-ambiguity': {'acc': 0.8906304423545803,
  'jsd': 0.24911370497669294,
  'kld': 0.5053047694248999},
 'high-ambiguity': {'acc': 0.6725775807473084,
  'jsd': 0.32948282138184765,
  'kld': 0.8568234905840153}}

### Evaluate the probability distribution performance
This model has been trained on the training dataset using gold labels, fine tuned on the dev dataset using probability distributions, and evaluated on the test dataset using the probability distributions.

In [15]:
eval_df = pd.read_json(f'{BASE_OUTPUT_PATH}/trained_model_snli_baseline/trained_train_gold_dev_probs_evaled_test_probs/eval_predictions.jsonl', lines=True)
eval_df = calculate_example_metrics(eval_df)
generate_all_metrics_dict(eval_df)

{'all': {'acc': 0.8907776872964169,
  'jsd': 0.21193841990962692,
  'kld': 0.25899654742502876},
 'no-ambiguity': {'acc': 0.9538518794194268,
  'jsd': 0.18520251934684548,
  'kld': 0.16976154564140516},
 'medium-ambiguity': {'acc': 0.8857540926506444,
  'jsd': 0.22325239907851072,
  'kld': 0.30014324071045295},
 'high-ambiguity': {'acc': 0.6852438252058265,
  'jsd': 0.28236039294985604,
  'kld': 0.48788618844006404}}